# Segundo trabalho - Programação Paralela e Concorrente

### Membros: 	Silvio Willian Caires Batista, André Vitor Santana Souza

## Polícia e Ladrão Online - Remake Visual

# Introdução
Polícia e Ladrão Online foi um pequeno projeto feito para uma atividade extra nesta matéria. Para este trabalho final, o grupo, juntamente com o professor, decidiu melhorar o jogo, otimizando e melhorando a experiência gráfica.


# O que foi feito
Primeiramente foi estudada a biblioteca “pygames” que permitiu uma melhor experiência visual e fácil controle dos eventos do teclado (movimentação). Feito isso, foi alterada a estrutura do servidor para comportar informações dos jogadores que agora eram dependentes dos objetos gerados por esta biblioteca. Por fim, foram feitos testes com diversas disposições de paralelismo de gpu e cpu, a fim de buscar a melhor configuração para este sistema. Com esses resultados em mão, foram feitos gráficos e cálculos para chegar na versão final do jogo, que atualmente conta apenas com um policial e um ladrão por campo.


# Como foi feito
Para a utilização do “pygames” substituímos o modelo de matrizes pelos objetos do próprio pygames. O ladrão pertence a um grupo chamado “ladroes”, e o policial a um grupo chamado “policiais”, o mesmo para as moedas, obstáculos semi-transponíveis e obstáculos intransponíveis. No lado do servidor, é guardado um array de “rect” - que é a posição - dos jogadores (polícia ou ladrão), uma variável que indica qual foi a última moeda capturada, e algumas variáveis de controle.

In [ ]:
def __init__(self):
    self.jogadores = []
    self.exposed_Killmoeda = ""
    self.exposed_fim = False
    self.jogador = -1
    self.ladroes = 0
    self.exposed_pos_x = [620/31, (620 / 31) * 29]
    self.exposed_pos_y = [261/13, (261 / 13) * 11]
    self.exposed_x = [1,29]
    self.exposed_y = [1,11]

Após o jogador se conectar ao banco, ele solicita seu index neste array, e sua índole. O servidor então retorna seu index e a índole de acordo com o número de jogadores existentes.

### Jogador:

In [ ]:
conn = rpyc.connect("localhost", 18861)
indole = conn.root.getIndole()
if indole == 'ladrao':
    jog1 = Objetos.ladrao(pos_x,pos_y)
    pygame.key.set_repeat(1, 150)
else:
    jog1 = Objetos.policia(pos_x,pos_y)
    pygame.key.set_repeat(1, 50)
print(indole)
indice = conn.root.getCredenciais(jog1.rect, indole)
print(indice)

### Servidor:

In [ ]:
print(len(self.jogadores))
if (len(self.jogadores) % 2 == 1):
    return "policia"
else:
    self.ladroes += 1
return "ladrao"
                            

Depois o jogo entra em loop (while no fim) esperando um evento do teclado (setas) para movimentar o jogador, ao perceber um evento de movimento, o sistema identifica a índole do jogador e verifica se sua movimentação é possível e caso seja, executa ela. Após executar a movimentação local, chama-se uma função que atualiza a posição do jogador no servidor. Faz o mesmo caso o jogador seja um ladrão e tenha passado por uma moeda. No final do loop, atualiza na tela as posições de todos os jogadores, também verifica se algum ladrão foi capturado.

### Jogador:

In [ ]:
global fim

    while not fim:

        inicio = time.time()
        for event in pygame.event.get():
            pygame.key.get_repeat()
            if event.type == pygame.QUIT:
                fim = True
            if event.type == pygame.KEYDOWN:
                if(indole == 'ladrao'):
                    if event.key == pygame.K_UP and not (pos_y - 261/13) <= 0 and not blockl(px,py-1,campo):
                        jog1.rect = jog1.rect.move(up)
                        py = py - 1
                        pos_y = pos_y - 261/13
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_DOWN and not(pos_y + 261/13 > 261 - (2*261/13)) and not blockl(px,py+1,campo):
                        jog1.rect = jog1.rect.move(down)
                        py = py + 1
                        pos_y = pos_y + 261/13
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_RIGHT and not(pos_x  >= 620 - (2*620/31)) and not blockl(px+1,py,campo):
                        jog1.rect = jog1.rect.move(dir)
                        px = px + 1
                        pos_x = pos_x + 620/31
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_LEFT and not (pos_x - 620/31 <  620/31) and not blockl(px-1,py,campo):
                        jog1.rect = jog1.rect.move(esq)
                        px = px - 1
                        pos_x = pos_x - 620/31
                        conn.root.putJogador(indice, jog1.rect)
                else:
                    if event.key == pygame.K_UP and not (pos_y - 261/13) <= 0 and not block(px,py-1,campo):
                        jog1.rect = jog1.rect.move(up)
                        py = py - 1
                        pos_y = pos_y - 261/13
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_DOWN and not(pos_y + 261/13 > 261 - (2*261/13)) and not block(px,py+1,campo):
                        jog1.rect = jog1.rect.move(down)
                        py = py + 1
                        pos_y = pos_y + 261/13
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_RIGHT and not(pos_x  >= 620 - (2*620/31)) and not block(px+1,py,campo):
                        jog1.rect = jog1.rect.move(dir)
                        px = px + 1
                        pos_x = pos_x + 620/31
                        conn.root.putJogador(indice, jog1.rect)
                    if event.key == pygame.K_LEFT and not (pos_x - 620/31 <  620/31) and not block(px-1,py,campo):
                        jog1.rect = jog1.rect.move(esq)
                        px = px - 1
                        pos_x = pos_x - 620/31
                        conn.root.putJogador(indice, jog1.rect)
        #Verifica se o jogador passou por uma moeda
        #O servidor tem q fazer isso verificando
        #se algum ladrao passou por moeda

        a = Thread(target=verificaMoedas(moedas, conn))
        a.start()

        b = Thread(target=verificaKillMoeda(moedas, conn, jog1, indole, cont))
        b.start()

        c = Thread(target=verificaKillLadrao(conn, indice, indole, derrota))
        c.start()

        #Desenho da tela
        screen.fill(black)
        screen.blit(campo2, camporect)
        for moeda in moedas:
            screen.blit(moeda.image,moeda.rect)
        screen.blit(jog1.image, jog1.rect)
        jogadores = conn.root.getJogadores()
        for jogador in jogadores:
            if(jogador['vivo']):
                if(jogador['indole'] == 'ladrao'):
                    screen.blit(ladrao, jogador['posicao'])
                else:
                    screen.blit(policia, jogador['posicao'])
            else:
                print("errou")
        pygame.display.flip()
        relogio.tick(10)
        fim1 = time.time()
        print( (fim1 - inicio)*1000)

In [ ]:
#Server:
def exposed_putJogador(self, indice, new):
   try:
       self.jogadores[indice]['posicao'] = new
       return True
   except:
       return False

Para fazer os testes de desempenho, foi colocado o time no início e no fim do loop, para saber quanto tempo foi gasto para realizar cada atualização. Com esses dados, foi feito três tabelas para cada configuração, uma mostrando o tempo gasto para a atualização quando não há movimentação, uma para quando há a movimentação e uma para quando há a movimentação e uma moeda é pega.
# Testes de desempenho

O primeiro teste foi com o programa sequencial. No gráfico a seguir está as medidas de tempo de atualização da tela para o personagem parado (Tempo Estático (ms)) e para o personagem andando (Tempo Movimento (ms)). Não houve diferença no tempo de atualização nos dois cenários.


 <img src="images/desempenho1.png" width="640" height="480" align="center">